# Tutorial for MUR SST on AWS  

(parts pf this tutorial were adapted from pangeo's ocean_ssh_example.ipynb by Dr. Ryan Abernathey)
  
Credits:

tutorial development

* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA
* [Dr. Rich Signell](mailto:rsignell@usgs.gov) - USGS

creating of the Zarr MUR SST dataset

* [Aimee Barciauskas](mailto:aimee@developmentseed.org) - Development Seed
* [Dr. Rich Signell](mailto:rsignell@usgs.gov) - USGS
* [Dr. Chelle Gentemann](mailto:gentemann@faralloninstitute.org)    - Farallon Institute, USA
-------------


# Structure of this tutorial

1. Opening data
2. Data plotting and exploration

# 1. Opening data

-------------------

## Import python packages

You are going to want to turn off warnings and set xarray display options

In [ ]:
import warnings
import numpy as np
import pandas as pd
import xarray as xr
import fsspec

warnings.simplefilter('ignore') # filter some warning messages
xr.set_options(display_style="html")  #display dataset nicely 

### start a cluster, the key to reading effectively on Cloud

- This will set up a cluster for you and give you a path that you can paste into the top of the Dask dashboard to visualize parts of your cluster.  You don't need to paste it into the Dask dashboard for this to work.

In [ ]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client

In [ ]:
cluster = KubeCluster(n_workers=60)
client = Client(cluster)
cluster

** ☝️ Don’t forget to click the link above or copy it to the Dask dashboard to view the scheduler dashboard! **

### Initialize Dataset

Here we load the dataset from the zarr store. Note that this very large dataset initializes nearly instantly, and we can see the full list of variables and coordinates.

In [ ]:
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)

### Examine Metadata

For those unfamiliar with this dataset, the variable metadata is very helpful for understanding what the variables actually represent
Printing the dataset will show you the dimensions, coordinates, and data variables with clickable icons at the end that show more metadata and size.

In [ ]:
ds_sst

# 2. Data plotting and exploration

``xarray`` plotting functions rely on matplotlib internally, but they make use of all available metadata to make the plotting operations more intuitive and interpretable.  
More plotting examples are given [here](http://xarray.pydata.org/en/stable/plotting.html)

### Here we use ``holoviews`` and ``hvplot`` thing using interactive graphics

In [ ]:
import hvplot.xarray
import holoviews as hv
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

#### Let's explore the data

- look at all the SST data
- look at the SST data masked to only ocean and ice-free data

In [ ]:
sst = ds_sst['analysed_sst']
sst_masked = ds_sst['analysed_sst'].where((ds_sst.mask==1) & ((ds_sst.sea_ice_fraction<.15) | np.isnan(ds_sst.sea_ice_fraction)))

### Plot a timeseries in the 2015 Blob Region

In [ ]:
sst.sel(lon=-140, lat=53).hvplot(grid=True)

### Plot a global image of SST on 10/1/2015

In [ ]:
%%time
sst.sel(time='2015-10-01').hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, cmap='rainbow', tiles='EsriImagery')

### Plot a global image of SST, only ocean and ice-free on 10/1/2015

- With all data, it is important to explore it and understand what is contains before doing an analysis.
- The ice mask used by MUR SST is from NSIDC and is based on satellite passive microwave estimates of sea ice concentration
- The satellite data isn't available near land, so the is no estimate of sea ice concentration near land
- For this data, it means that there are some erroneous SSTs near land, that is likely ice and this is something to be aware of.

In [ ]:
%%time
sst_masked.sel(time='2015-10-01',lat=slice(-70,0)).hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, cmap='rainbow', tiles='EsriImagery')

### Calculate the SST anomaly by subtracting a daily climatology

In [ ]:
%%time
sst_monthly = sst_masked.resample(time='1MS').mean('time',keep_attrs=True,skipna=False)
climatology_mean = sst_monthly.groupby('time.month').mean('time',keep_attrs=True,skipna=False)
anom_ts = sst_monthly.groupby('time.month')-climatology_mean  #take out annual mean to remove trends

#### Show the dimensions of the SST anomaly

In [ ]:
anom_ts

### Plot the anomaly on 10/1/2015

In [ ]:
%%time
anom_ts.sel(time='2015-10-01',lat=slice(-60,70)).hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, clim=(-2,2), cmap='seismic', tiles='EsriImagery')

### Plot the anomaly at the same location as the timeseries above

- THIS IS THE LINE THAT DOESN'T WORK

In [ ]:
#anom_ts.sel(lon=-140, lat=53).hvplot(grid=True)

### Subset the El Niño/La Niña Region:

In [ ]:
%%time
sst_elnino = sst.sel(lon=slice(-180,-70), lat=slice(-25,25))

### Difference the monthly mean temperature fields for Jan 2016 (El Niño) and Jan 2014 (normal)

In [ ]:
%%time
sst_jan2016 = sst_elnino.sel(time=slice('2016-01-01','2016-02-01')).mean(dim='time')
sst_jan2014 = sst_elnino.sel(time=slice('2014-01-01','2014-02-01')).mean(dim='time')

In [ ]:
%%time
(sst_jan2016 - sst_jan2014).hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, cmap='rainbow', tiles='EsriImagery')

### Plotting on maps

For plotting on maps, we rely on the excellent [cartopy](http://scitools.org.uk/cartopy/docs/latest/index.html) library.

In [ ]:
import cartopy.crs as ccrs

### In cartopy you need to define the map projection you want to plot.  

- Common ones are Ortographic and PlateCarree.
- You can add coastlines and gridlines to the axes as well.

### Create a monthly average SST from 2015 then image it

In [ ]:
sst_2015_monthly = sst.sel(time=slice('2015-01-01','2016-01-01')).groupby('time.month').mean(dim='time')

In [ ]:
sst_2015_monthly.hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, cmap='rainbow', projection=ccrs.Orthographic(-80, 35))

### Add coastline

In [ ]:
sst_2015_monthly.sel(month=10).hvplot.quadmesh(x='lon', y='lat', geo=True, 
                rasterize=True, cmap='rainbow', projection=ccrs.Orthographic(-80, 35),
                                           coastline='110m')

### I want to print out an image and the hvplot needs an additional library so I thought I would try using matplotlib, but it isn't working either.....

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
datamin = 0
datamax = 30

ax = plt.axes(projection=ccrs.Orthographic())  #here is where you set your axis projection

(sst_2015_monthly.sel(month=10)-273.15).plot(ax=ax, 
                                  transform=ccrs.PlateCarree(),  #set data projection
                                  vmin=datamin,                  #data min
                                  vmax=datamax)                  #data min

#ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())  #data projection
ax.coastlines(resolution='50m')

## A nice cartopy tutorial is [here](http://earthpy.org/tag/visualization.html)

# xarray can do more!

* concatentaion
* open network located files with openDAP
* import and export Pandas DataFrames
* .nc dump to 
* groupby_bins
* resampling and reduction

For more details, read this blog post: http://continuum.io/blog/xray-dask


## Where can I find more info?

### For more information about xarray

- Read the [online documentation](http://xarray.pydata.org/)
- Ask questions on [StackOverflow](http://stackoverflow.com/questions/tagged/python-xarray)
- View the source code and file bug reports on [GitHub](http://github.com/pydata/xarray/)

### For more doing data analysis with Python:

- Thomas Wiecki, [A modern guide to getting started with Data Science and Python](http://twiecki.github.io/blog/2014/11/18/python-for-data-science/)
- Wes McKinney, [Python for Data Analysis](http://shop.oreilly.com/product/0636920023784.do) (book)

### Packages building on xarray for the geophysical sciences


- [eofs](https://github.com/ajdawson/eofs): empirical orthogonal functions by Andrew Dawson
- [infinite-diff](https://github.com/spencerahill/infinite-diff) by Spencer Hill 
- [aospy](https://github.com/spencerahill/aospy) by Spencer Hill and Spencer Clark
- [regionmask](https://github.com/mathause/regionmask) by Mathias Hauser
- [salem](https://github.com/fmaussion/salem) by Fabien Maussion

Resources for teaching and learning xarray in geosciences:
- [Fabien's teaching repo](https://github.com/fmaussion/teaching): courses that combine teaching climatology and xarray
